#**스마트폰 센서 데이터 기반 모션 분류**
# 단계2 : 기본 모델링


## 0.미션

* 데이터 전처리
    * 가변수화, 데이터 분할, NaN 확인 및 조치, 스케일링 등 필요한 전처리 수행
* 다양한 알고리즘으로 분류 모델 생성
    * 최소 4개 이상의 알고리즘을 적용하여 모델링 수행
    * 성능 비교
    * 각 모델의 성능을 저장하는 별도 데이터 프레임을 만들고 비교
* 옵션 : 다음 사항은 선택사항입니다. 시간이 허용하는 범위 내에서 수행하세요.
    * 상위 N개 변수를 선정하여 모델링 및 성능 비교
        * 모델링에 항상 모든 변수가 필요한 것은 아닙니다.
        * 변수 중요도 상위 N개를 선정하여 모델링하고 타 모델과 성능을 비교하세요.
        * 상위 N개를 선택하는 방법은, 변수를 하나씩 늘려가며 모델링 및 성능 검증을 수행하여 적절한 지점을 찾는 것입니다.

## 1.환경설정

### (1) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.




* 함수 생성

In [2]:
# 변수의 특성 중요도 계산하기
def plot_feature_importance(importance, names, result_only = False, topn = 'all'):
    feature_importance = np.array(importance)
    feature_name = np.array(names)

    data={'feature_name':feature_name,'feature_importance':feature_importance}
    fi_temp = pd.DataFrame(data)

    #변수의 특성 중요도 순으로 정렬하기
    fi_temp.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_temp.reset_index(drop=True, inplace = True)

    if topn == 'all' :
        fi_df = fi_temp.copy()
    else :
        fi_df = fi_temp.iloc[:topn]

    #변수의 특성 중요도 그래프로 그리기
    if result_only == False :
        plt.figure(figsize=(10,20))
        sns.barplot(x='feature_importance', y='feature_name', data = fi_df)

        plt.xlabel('importance')
        plt.ylabel('feature name')
        plt.grid()

    return fi_df

### (2) 데이터 불러오기

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용
* 세부 요구사항
    - 전체 데이터 'data01_train.csv' 를 불러와 'data' 이름으로 저장합니다.
        - data에서 변수 subject는 삭제합니다.
    - 데이터프레임에 대한 기본 정보를 확인합니다.( .head(), .shape 등)

#### 1) 데이터 로딩

In [4]:
data = pd.read_csv('/content/drive/MyDrive/에이블스쿨_미니프로젝트/Mini_Project5_2/data01_train.csv')

In [5]:
data.drop('subject',axis=1,inplace=True)

In [6]:
data.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.487737,-0.816696,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163,STANDING
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.237820,-0.693515,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944,LAYING
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,-0.535287,-0.829311,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755,STANDING
3,0.289795,-0.035536,-0.150354,-0.231727,-0.006412,-0.338117,-0.273557,0.014245,-0.347916,0.008288,...,-0.004012,-0.408956,-0.255125,0.612804,0.747381,-0.072944,-0.695819,0.287154,0.111388,WALKING
4,0.394807,0.034098,0.091229,0.088489,-0.106636,-0.388502,-0.010469,-0.109680,-0.346372,0.584131,...,-0.157832,-0.563437,-0.044344,-0.845268,-0.974650,-0.887846,-0.705029,0.264952,0.137758,WALKING_DOWNSTAIRS


In [7]:
data.shape

(5881, 562)

#### 2) 기본 정보 조회

In [8]:
data.describe()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
count,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,...,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000
mean,0.274811,-0.017799,-0.109396,-0.603138,-0.509815,-0.604058,-0.628151,-0.525944,-0.605374,-0.465490,...,0.126955,-0.305883,-0.623548,0.008524,-0.001185,0.009340,-0.007099,-0.491501,0.059299,-0.054594
std,0.067614,0.039422,0.058373,0.448807,0.501815,0.417319,0.424345,0.485115,0.413043,0.544995,...,0.249176,0.322808,0.310371,0.339730,0.447197,0.608190,0.476738,0.509069,0.297340,0.278479
min,-0.503823,-0.684893,-1.000000,-1.000000,-0.999844,-0.999667,-1.000000,-0.999419,-1.000000,-1.000000,...,-0.965725,-0.979261,-0.999765,-0.976580,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.980143
25%,0.262919,-0.024877,-0.121051,-0.992774,-0.977680,-0.980127,-0.993602,-0.977865,-0.980112,-0.936067,...,-0.021610,-0.541969,-0.845985,-0.122361,-0.294369,-0.481718,-0.373345,-0.811397,-0.018203,-0.141555
50%,0.277154,-0.017221,-0.108781,-0.943933,-0.844575,-0.856352,-0.948501,-0.849266,-0.849896,-0.878729,...,0.133887,-0.342923,-0.712677,0.010278,0.005146,0.011448,-0.000847,-0.709441,0.182893,0.003951
75%,0.288526,-0.010920,-0.098163,-0.242130,-0.034499,-0.262690,-0.291138,-0.068857,-0.268539,-0.013690,...,0.288944,-0.127371,-0.501158,0.154985,0.285030,0.499857,0.356236,-0.511330,0.248435,0.111932
max,1.000000,1.000000,1.000000,1.000000,0.916238,1.000000,1.000000,0.967664,1.000000,1.000000,...,0.946700,0.989538,0.956845,1.000000,1.000000,0.998702,0.996078,0.977344,0.478157,1.000000


## **2. 데이터 전처리**

* 가변수화, 데이터 분할, NaN 확인 및 조치, 스케일링 등 필요한 전처리를 수행한다.


### (1) 데이터 분할1 : x, y

* 세부 요구사항
    - x, y로 분할합니다.

In [9]:
from sklearn.model_selection import train_test_split
target='Activity'
X=data.drop(target,axis=1)
y=data[target]

### (2) 스케일링(필요시)


* 세부 요구사항
    - 스케일링을 필요로 하는 알고리즘 사용을 위해서 코드 수행
    - min-max 방식 혹은 standard 방식 중 한가지 사용.

### (3) 데이터분할2 : train, validation

* 세부 요구사항
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [10]:
x_train,x_val,y_train,y_val=train_test_split(X,y,test_size=0.2)

## **3. 기본 모델링**



* 세부 요구사항
    - 최소 4개 이상의 알고리즘을 적용하여 모델링을 수행한다.
    - 각 알고리즘별로 전체 변수로 모델링, 상위 N개 변수를 선택하여 모델링을 수행하고 성능 비교를 한다.
    - (옵션) 알고리즘 중 1~2개에 대해서, 변수 중요도 상위 N개를 선정하여 모델링하고 타 모델과 성능을 비교.
        * 상위 N개를 선택하는 방법은, 변수를 하나씩 늘려가며 모델링 및 성능 검증을 수행하여 적절한 지점을 찾는 것이다.

### (1) 알고리즘1 :

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
lr = LogisticRegression(max_iter=1500)
lr.fit(x_train,y_train)
lr_pred = lr.predict(x_val)
print(classification_report(y_val,lr_pred))

                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       240
           SITTING       0.94      0.94      0.94       219
          STANDING       0.94      0.94      0.94       207
           WALKING       1.00      1.00      1.00       200
WALKING_DOWNSTAIRS       0.99      1.00      1.00       148
  WALKING_UPSTAIRS       1.00      0.99      1.00       163

          accuracy                           0.98      1177
         macro avg       0.98      0.98      0.98      1177
      weighted avg       0.98      0.98      0.98      1177



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### (2) 알고리즘2 :

In [16]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
y_train_la = label.fit_transform(y_train)
y_val_la = label.transform(y_val)

In [17]:
from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(x_train, y_train_la)
xgb_pred=xgb.predict(x_val)
print(classification_report(y_val_la,xgb_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       240
           1       0.99      0.96      0.97       219
           2       0.96      0.99      0.97       207
           3       0.99      0.99      0.99       200
           4       1.00      0.99      1.00       148
           5       0.99      1.00      1.00       163

    accuracy                           0.99      1177
   macro avg       0.99      0.99      0.99      1177
weighted avg       0.99      0.99      0.99      1177



### (3) 알고리즘3 :

In [28]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()
sgd.fit(x_train,y_train)
sgd_pred=sgd.predict(x_val)
print(classification_report(y_val,sgd_pred))

                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       240
           SITTING       0.93      0.97      0.95       219
          STANDING       0.96      0.92      0.94       207
           WALKING       1.00      1.00      1.00       200
WALKING_DOWNSTAIRS       1.00      0.98      0.99       148
  WALKING_UPSTAIRS       0.98      1.00      0.99       163

          accuracy                           0.98      1177
         macro avg       0.98      0.98      0.98      1177
      weighted avg       0.98      0.98      0.98      1177



In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
models = [
    LogisticRegression(),
    GaussianNB(),
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SVC(),
    MLPClassifier()
]

for model in models:
    model.fit(x_train, y_train)
    y_pred=model.predict(x_val)
    accuracy = accuracy_score(y_val,y_pred)
    print(f'{model.__class__.__name__}: Accuracy = {accuracy}')

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression: Accuracy = 0.9796091758708582
GaussianNB: Accuracy = 0.7952421410365336
KNeighborsClassifier: Accuracy = 0.9558198810535259
DecisionTreeClassifier: Accuracy = 0.9328802039082413
RandomForestClassifier: Accuracy = 0.9753610875106202
SVC: Accuracy = 0.9651656754460493
MLPClassifier: Accuracy = 0.9847068819031436


In [32]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 2.6 MB/s eta 0:00:00


In [33]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier()
cat.fit(x_train,y_train_la)
cat_pred = cat.predict(x_val)
accuracy=accuracy_score(y_val_la, cat_pred)
print(f'Accuracy = {accuracy}')

Learning rate set to 0.085629
0:	learn: 1.5151248	total: 1.32s	remaining: 21m 56s
1:	learn: 1.3264357	total: 2.08s	remaining: 17m 15s
2:	learn: 1.1725195	total: 2.9s	remaining: 16m 4s
3:	learn: 1.0604203	total: 3.59s	remaining: 14m 54s
4:	learn: 0.9655821	total: 4.79s	remaining: 15m 52s
5:	learn: 0.8878605	total: 6.18s	remaining: 17m 3s
6:	learn: 0.8153884	total: 7.48s	remaining: 17m 41s
7:	learn: 0.7536573	total: 8.99s	remaining: 18m 35s
8:	learn: 0.6988370	total: 11.7s	remaining: 21m 30s
9:	learn: 0.6539807	total: 14.3s	remaining: 23m 31s
10:	learn: 0.6095418	total: 16.3s	remaining: 24m 28s
11:	learn: 0.5715079	total: 17.7s	remaining: 24m 13s
12:	learn: 0.5374288	total: 19s	remaining: 24m
13:	learn: 0.5067520	total: 20.1s	remaining: 23m 32s
14:	learn: 0.4808169	total: 21.3s	remaining: 23m 18s
15:	learn: 0.4540639	total: 22.6s	remaining: 23m 7s
16:	learn: 0.4312815	total: 23.8s	remaining: 22m 54s
17:	learn: 0.4087546	total: 25s	remaining: 22m 43s
18:	learn: 0.3884591	total: 26.2s	rema

### (4) 알고리즘4 :

In [36]:
!pip install --upgrade pip
!pip install pycaret
# from pycaret.classification import setup
# setup(data = x_train, target = y_train_la)
# from pycaret.classification import compare_models
# compare_models()

,Description,Value
0,Session id,2148
1,Target,target
2,Target type,Multiclass
3,Original data shape,"(4704, 562)"
4,Transformed data shape,"(4704, 562)"
5,Transformed train set shape,"(3292, 562)"
6,Transformed test set shape,"(1412, 562)"
7,Numeric features,561
8,Preprocess,True
9,Imputation type,simple


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,07:17:31
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9827,0.9991,0.9827,0.9829,0.9827,0.9792,0.9792,11.6410
ridge,Ridge Classifier,0.9778,0.0000,0.9778,0.9781,0.9778,0.9733,0.9734,0.2300
rf,Random Forest Classifier,0.9687,0.9990,0.9687,0.9690,0.9687,0.9624,0.9624,4.5350
svm,SVM - Linear Kernel,0.9572,0.0000,0.9572,0.9646,0.9564,0.9485,0.9504,0.4130
knn,K Neighbors Classifier,0.9426,0.9931,0.9426,0.9435,0.9423,0.9309,0.9312,0.2060
dt,Decision Tree Classifier,0.9234,0.9541,0.9234,0.9247,0.9234,0.9079,0.9081,1.7520
nb,Naive Bayes,0.7625,0.9582,0.7625,0.7953,0.7562,0.7144,0.7239,0.1310
ada,Ada Boost Classifier,0.5425,0.8793,0.5425,0.3466,0.4062,0.4428,0.5025,11.1260
qda,Quadratic Discriminant Analysis,0.3594,0.6143,0.3594,0.2960,0.2282,0.2281,0.2807,0.8240


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
from pycaret.classification import setup
setup(data = x_train, target = y_train_la)
from pycaret.classification import compare_models
compare_models()